In [11]:
import docx2txt
import string

In [12]:
tmp_lines = []
def isAllCaps(line):
    for ch in line:
        if(ch.islower()):
            return False
    return True
def isActionWithOneLinebreak(line):
    if line == "":
        return True
    if line[0] =='(' or line[-1] ==')':
        return False
    if not isName(line):
        return True
        
    return False
def checkNext3Lines(i,lines):
    return (isActionWithOneLinebreak(lines[i+1]) and 
    isActionWithOneLinebreak(lines[i+2]) and
    isActionWithOneLinebreak(lines[i+3]))
def checkPrevious3Lines(i,lines):
    return (isActionWithOneLinebreak(lines[i-1]) and 
    isActionWithOneLinebreak(lines[i-2]) and
    isActionWithOneLinebreak(lines[i-3]) and
    not isName(lines[i-4]))


    
def isAction(i,lines):
    #print("i-1,i-3=>",lines[i-1] , lines[i-2] , lines[i-3])
    #if 5050>i>5000:
    #    print("try.. ",i,lines[i])
    if(lines[i] == "" or i<4):
        return False
    if(i+3>=len(lines)):
        return False
    if("--" in lines[i]):
        return False
    if lines[i][0].islower():
        return False
    if lines[i][-1] == '?':
        return False
    if lines[i] == ' ' or lines[i] == '\t':
        return True
    if('(' is lines[i][0] or ')' is lines[i][-1]):
        return False
    if(isAllCaps(lines[i])):
        return False
    if(checkPrevious3Lines(i,lines)):
        if(checkNext3Lines(i,lines)):
            if(len(lines[i])<3):
                print('-',len(lines[i]),'-',lines[i],'-')
            if(not lines[i][1].isupper()):
    #            if 5050>i>5000:
    #                print("try.. ",i,lines[i])
                if(lines[i-4] != "" and lines[i-4][0] == '('):
                    return False
                if(lines[i-4] == ""):
                    return True
                return True
    if "EXT." in lines[i-4] or "INT." in lines[i-4]:
        return True
    return False
def isName(name):
    if not isAllCaps(name):
        return False
    if "INT." in name or "EXT." in name:
        return False
    if len(name)<3:
        return False
    return True
#todo:checks whether the file has errors or not
def errorInFile(lines):
    for line in lines:
        if(bool("(" in line) != bool(")" in line)):
            pass
            #print("(" in line , ")" in line)
            #print(line)
            #return True
    return False

In [26]:
MOVIE_NAME = ""
DOCX_NAME = "FightBelle" #without extension
my_text = docx2txt.process(DOCX_NAME +".docx")
lines = my_text.split('\n')
out = open(DOCX_NAME + "_out.txt","w")
txtfile = open(DOCX_NAME + "_modified.txt","w")
orgfile = open(DOCX_NAME+ "_original.txt","w")
print("Movie:",MOVIE_NAME,"Screenplay:",DOCX_NAME)
discovery = 1

if(errorInFile(lines)):
    print("ERROR IN FILE")
    exit(1)
tmp_lines = []
#remove tabs,(MORE),pagenumbers from text    
print("Modifing the file...")
for i,line in enumerate(lines):
    orgfile.write(lines[i]+'\n')
for i,line in enumerate(lines):
    lines[i] = lines[i].replace('(MORE)','')#replace all (MORE)
    if(lines[i]==''):
        continue
    lines[i] = lines[i].replace('\t',' ')#replace all tab characters with space
for i,line in enumerate(lines):
    #print(lines[i])
    if(lines[i]==''):
        continue
    if(lines[i][0] == ' '):
        #print("tab:",lines[i])
        lines[i] = lines[i][1:]
        #print("aft:",lines[i])
    if(2<=len(lines[i])<=5 and lines[i][-1] == '.' and  lines[i][:-1].isdigit()):#replace pagenumbers
        #print("page:",lines[i])
        lines[i] = ""
    if(lines[i] != "" and ')' in lines[i] and '(' not in lines[i]):#find parantheticals that occupy more than oneline,merge them
        open_id = i
        #print("=>",lines[i-1],lines[i])
        while('(' not in lines[open_id]):
            open_id -= 1
        new_line = lines[open_id]+" "+lines[i]
        #print(new_line)
        del lines[open_id:i+1]
        lines.insert(open_id,new_line)
        
        
test = 988
#print("%".join(lines[test-10:test+10]))
print("Remove unnecassary action(direction)s")

for i,line in enumerate(lines):
    #remove action(direction)'s and 3 '\n' after them
    if(isAction(i,lines)):
        print(i,lines[i])
        #del lines[i:i+4]
        
    else:
        tmp_lines.append(lines[i])
    #if i > 1000:
     #   break
lines = tmp_lines
#print modified file to check
for i,line in enumerate(lines):
    txtfile.write(line+"\n")
print("Parsing..")
for i,line in enumerate(lines):
    if('(' in line):
        if("CONT'D" not in line and "O.S." not in line and "V.O" not in line and "MORE" not in line):
            pre_emotion_empty_count = 0
            pre_step = 0
            pre_name_count = 0
            result = []
        
            while(pre_name_count < 1 or len(lines[i - pre_step]) < 3 or  not  isAllCaps(lines[i-pre_step])):
                if isAllCaps(lines[i-pre_step]) and len(lines[i-pre_step]) > 0:
                    pre_name_count += 1
                pre_step += 1
            if("INT." in lines[i-pre_step] or "EXT." in lines[i-pre_step]):
                result.append("")
                result.append("")
                result.append("")
                pre_step-=1
                while(lines[i-pre_step] == ""):
                    pre_step-=1
                
            post_emotion_empty_count = 0
            post_step = 0
            post_name_count = 0

            while(post_name_count < 2 or len(lines[i + post_step]) < 3 or  not  isAllCaps(lines[i+post_step]) ):
                if "EXT." in lines[i+post_step] or "INT." in lines[i+post_step] :
                    break
                if isAllCaps(lines[i+post_step]) and len(lines[i+post_step]) > 0:
                    post_name_count += 1
                post_step += 1
            while "EXT." in lines[i+post_step] or "INT." in lines[i+post_step]:
                post_step -=1
            
            #print(lines[i-pre_step:i+post_step])
            #print(pre_step,post_step)
            printed_lines = lines[i-pre_step:i+post_step]
            
            for line_in_range in lines[i-pre_step:i+post_step]:
                if(line_in_range!="" and not line_in_range[:-1].isnumeric()):
                    if(line_in_range!="(O.S.)"):
                        if(len(result)%3 == 1 and result[-1][0] != '(' and line_in_range[0]!='('):
                            result.append("")
                        result.append(line_in_range)
            name_utterance = []
            tmp_utterance = "%%%"
            for x in result:
                if(isName(x)):
                    if(tmp_utterance != "%%%"):
                        name_utterance.append(tmp_utterance)
                    tmp_utterance = ""
                    name_utterance.append(x)
                else:
                    if(x!=""):
                        tmp_utterance +=" "
                    tmp_utterance +=""+ x
                
            if(tmp_utterance != ""):
                name_utterance.append(tmp_utterance)
            #print("=".join(name_utterance))
            out.write("\t".join(name_utterance)+"\n")
            #print("+".join(result))
            #out.write("\t".join(result)+"\n")
            #print(discovery,i)
            discovery+=1
            #if(discovery>50):
             #   break
    elif(line.count("(")>2):
        print("Line {0} has more than one (",i)
out.close()
txtfile.close()
orgfile.close()
print("Done")


Movie:  Screenplay: FightBelle
Modifing the file...
Remove unnecassary action(direction)s
8 Aimee Parrott & Antonio Gangemi
82 Registered WGAw No. 1160416
94 Wedding GUESTS in hoop skirts and morning suits roam about.
102 Ten BRIDESMAIDS clad in fuchsia primp and gossip. A FLOWER GIRL frolics amidst balloons.
110 The room of the consummate beauty pageant contestant. In a white hoop skirt stands belle of the ball DELILAH MACINTOSH, early 20s.
114 Tending to the bride are: mother MIRABELLE, a domineering matriarch; bridesmaid CHARLENE, Delilah's older, heavier sister; and bridesmaid MARNEY, fetching and shallow.
126 Charlene pinches her, hard.
174 Delilah holds... then bursts into giggles.
200 She crouches to adjust Delilah's skirt.
224 She brushes past Charlene, who reddens.
324 Mirabelle gives Marney a look of disdain.
370 Charlene narrows her eyes at Delilah and marches out.
402 She meanders to the bureau. Ogles the program.
424 The bridegroom shadowboxes in front of a mirror. KELVIN 

In [22]:
print(' '.islower())

False
